# ESA Fire Climate Change Initiative

In [1]:
import netCDF4 as nc
import pandas as pd
import numpy as np
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')


In [2]:
fn = 'data/20160107-ESACCI-L4_FIRE-BA-MODIS-fv5.0.nc'
ds = nc.Dataset(fn)

In [3]:
list_variables = list()
for var in ds.variables:
    list_variables.append(var)
list_variables

['lat',
 'lat_bnds',
 'lon',
 'lon_bnds',
 'time',
 'time_bnds',
 'vegetation_class',
 'vegetation_class_name',
 'burned_area',
 'standard_error',
 'fraction_of_burnable_area',
 'fraction_of_observed_area',
 'number_of_patches',
 'burned_area_in_vegetation_class']

In [4]:
#explore variables
for var in list_variables: 
    array = ds[var][:]
    print(f" Variable {var} mit Shape {array.shape}")

 Variable lat mit Shape (720,)
 Variable lat_bnds mit Shape (720, 2)
 Variable lon mit Shape (1440,)
 Variable lon_bnds mit Shape (1440, 2)
 Variable time mit Shape (1,)
 Variable time_bnds mit Shape (1, 2)
 Variable vegetation_class mit Shape (18,)
 Variable vegetation_class_name mit Shape (18, 150)
 Variable burned_area mit Shape (1, 720, 1440)
 Variable standard_error mit Shape (1, 720, 1440)
 Variable fraction_of_burnable_area mit Shape (1, 720, 1440)
 Variable fraction_of_observed_area mit Shape (1, 720, 1440)
 Variable number_of_patches mit Shape (1, 720, 1440)
 Variable burned_area_in_vegetation_class mit Shape (1, 18, 720, 1440)


In [5]:
def create_column(dataset, dataframe, number_dimensions,name_variable):
    if (number_dimensions == 3):
        variable = dataset[name_variable][:]
        variable = variable[0]
        variable_flatt = variable.flatten()
        dataframe[name_variable] = variable_flatt
        return dataframe

In [6]:
lat = ds['lat'][:]
lon = ds['lon'][:]
lon

masked_array(data=[-179.875, -179.625, -179.375, ...,  179.375,  179.625,
                    179.875],
             mask=False,
       fill_value=1e+20,
            dtype=float32)

In [7]:
lons, lats = np.meshgrid(lon, lat)
lons

masked_array(
  data=[[-179.875, -179.625, -179.375, ...,  179.375,  179.625,  179.875],
        [-179.875, -179.625, -179.375, ...,  179.375,  179.625,  179.875],
        [-179.875, -179.625, -179.375, ...,  179.375,  179.625,  179.875],
        ...,
        [-179.875, -179.625, -179.375, ...,  179.375,  179.625,  179.875],
        [-179.875, -179.625, -179.375, ...,  179.375,  179.625,  179.875],
        [-179.875, -179.625, -179.375, ...,  179.375,  179.625,  179.875]],
  mask=False,
  fill_value=1e+20,
  dtype=float32)

(3) Runterbrechen der 2-dimensionalen Matrizen auf ein 1-dimensionalen Array. 

In [8]:
lons_flatt = lons.flatten()
lats_flatt = lats.flatten()
lons_flatt

masked_array(data=[-179.875, -179.625, -179.375, ...,  179.375,  179.625,
                    179.875],
             mask=False,
       fill_value=1e+20,
            dtype=float32)

In [9]:
#Variable burned_area_in_vegetation_class mit Shape (1, 18, 720, 1440)

burned_area_in_vegetation_class= ds['burned_area_in_vegetation_class'][:]
burned_area_in_vegetation_class = burned_area_in_vegetation_class[0]


In [10]:
whole_world = {
    'lon': lons_flatt, 
    'lat': lats_flatt, 
}
df_whole_world = pd.DataFrame(whole_world)
df_whole_world

,lon,lat
0,-179.875,89.875
1,-179.625,89.875
2,-179.375,89.875
3,-179.125,89.875
4,-178.875,89.875
...,...,...
1036795,178.875,-89.875
1036796,179.125,-89.875
1036797,179.375,-89.875
1036798,179.625,-89.875


In [11]:
df_whole_world = create_column(ds, df_whole_world, 3, 'burned_area')

## Vegetationsklassen 

In [12]:
vegetation_class = ds['vegetation_class'][:]
vegetation_class

masked_array(data=[ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110,
                   120, 130, 140, 150, 160, 170, 180],
             mask=False,
       fill_value=999999)

In [13]:
vegetation_class_name = ds['vegetation_class_name'][:]
#vegetation_class_name[1]

TODO Erklärung der Vegetationsklassen

In [14]:
name_vegetation_class ={
    10: ['no_data'], 
    20: ['cropland_rainfed'], #Ackerland 
    30: ['cropland_irrigated'], #Ackerland_bewässert
    40: ['50_mosaic_cropland_50_natural_vegetation'], #Ackerland >50%, natürliche Bepflanzung <50%
    50: ['tree_cover_broadleaved_evergreen'], # immergrüne Laubbäume 
    60: ['tree_cover_broadleaved_deciduous'], # laubabwerfend Laubbäume
    70: ['tree_cover_needleleaved_evergreen'], # immmergrüne Nadelbäume
    80: ['tree_cover_needleleaved_deciduous'], # laubabwerfend Laubbäume
    90: ['tree_cover_mixed_leave'], # Mischwald
    100: ['50_mosaic_tree_50_herbaceous'], # Bäume >50%, <50% krautig Bewachsen
    110: ['50_herbaceous_50_tree'], # >50% krautige Bewachsen, <50% Bäu,e 
    120: ['shrubland'], #Buschland
    130: ['grassland'], #Grasland
    140: ['lichens_and_mosses'], #Flechten und Moose
    150: ['sparse_vegetation'], #spärliche Vegetation 
    160: ['tree_cover_flooded_fresh_water'], #geflutete Bäume Süßwasser
    170: ['tree_cover_flooded_saline_water'], # geflutete Bäume Salzwasser
    180: ['shrub_flooded_water'] #geflutetes Buschland
}

Für das Projekt werden die einzelnen Kategorien zusammengefasst: 
* 10: no_data
* 20, 30, 40: cropland
* 100, 110: mosaic_tree_grass
* 120, 130, 140, 150: other_vegetation
* 160, 170, 180: flooded_area

Die Vegetationsklassen 50 -90 werden belassen, da die Untersuchung von Waldbränden im Fokus steht und dabei die genaue Aufsplitung interessant sein kann. 

In [15]:
i_vegetation_class = 10 
for burned_area_per_veg_class in burned_area_in_vegetation_class:
    s_vegetation_class= f"{i_vegetation_class}_burned_area"
    burned_area_per_veg_class_flatt= burned_area_per_veg_class.flatten()
    df_whole_world[s_vegetation_class] = burned_area_per_veg_class_flatt
    i_vegetation_class= i_vegetation_class + 10
    #print(a)

In [16]:
df_burned  = df_whole_world
df_burned

,lon,lat,burned_area,10_burned_area,20_burned_area,30_burned_area,40_burned_area,50_burned_area,60_burned_area,70_burned_area,...,90_burned_area,100_burned_area,110_burned_area,120_burned_area,130_burned_area,140_burned_area,150_burned_area,160_burned_area,170_burned_area,180_burned_area
0,-179.875,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-179.625,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-179.375,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-179.125,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-178.875,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1036795,178.875,-89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1036796,179.125,-89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1036797,179.375,-89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1036798,179.625,-89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
#Spalte umbennen
df_burned['cropland_burned_area'] = df_burned['20_burned_area'] + df_burned['30_burned_area'] + df_burned['40_burned_area']
df_burned['mosaic_tree_grass_burned_area'] = df_burned['100_burned_area'] + df_burned['100_burned_area']
df_burned['other_vegetation_burned_area']= df_burned['120_burned_area'] + df_burned['130_burned_area'] + df_burned['140_burned_area']+ df_burned['150_burned_area']
df_burned['flooded_area_burned_area']= df_burned['160_burned_area'] + df_burned['170_burned_area'] + df_burned['180_burned_area']


In [18]:
df_burned.drop(columns=['20_burned_area','30_burned_area','40_burned_area','100_burned_area','110_burned_area','120_burned_area', '130_burned_area', '140_burned_area', '150_burned_area','160_burned_area','170_burned_area','180_burned_area'], inplace=True)

In [19]:
replace_names={ '10_burned_area': name_vegetation_class[10][0] + "_burned_area",
                '50_burned_area': name_vegetation_class[50][0] + "_burned_area",
                '60_burned_area': name_vegetation_class[60][0] + "_burned_area", 
                '70_burned_area': name_vegetation_class[70][0] + "_burned_area", 
                '80_burned_area': name_vegetation_class[80][0] + "_burned_area", 
                '90_burned_area': name_vegetation_class[90][0] + "_burned_area", }
replace_names

{'10_burned_area': 'no_data_burned_area',
 '50_burned_area': 'tree_cover_broadleaved_evergreen_burned_area',
 '60_burned_area': 'tree_cover_broadleaved_deciduous_burned_area',
 '70_burned_area': 'tree_cover_needleleaved_evergreen_burned_area',
 '80_burned_area': 'tree_cover_needleleaved_deciduous_burned_area',
 '90_burned_area': 'tree_cover_mixed_leave_burned_area'}

In [20]:
df_burned.rename(columns = replace_names, inplace = True)
df_burned

,lon,lat,burned_area,no_data_burned_area,tree_cover_broadleaved_evergreen_burned_area,tree_cover_broadleaved_deciduous_burned_area,tree_cover_needleleaved_evergreen_burned_area,tree_cover_needleleaved_deciduous_burned_area,tree_cover_mixed_leave_burned_area,cropland_burned_area,mosaic_tree_grass_burned_area,other_vegetation_burned_area,flooded_area_burned_area
0,-179.875,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-179.625,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-179.375,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-179.125,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-178.875,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1036795,178.875,-89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1036796,179.125,-89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1036797,179.375,-89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1036798,179.625,-89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


###  Weitere Kennwerte

In [21]:
 #Variable standard_error mit Shape (1, 720, 1440)
 #Variable fraction_of_burnable_area mit Shape (1, 720, 1440)
 #Variable fraction_of_observed_area mit Shape (1, 720, 1440)
 #Variable number_of_patches mit Shape (1, 720, 1440)

Standard error: 
This value is the standard error of the estimation of BA in each grid cell, based on the aggregation of the confidence level of the pixel product.
Fraction of burnable area: 
Includes all land cover categories that can be burned. That means that it excludes water bodies, permanent snow and ice, urban areas and bare areas. Land cover information was extracted from the LC_cci project. 
Fraction of observed area: 
The fraction of observed area is included as a layer in the grid product with the particular aim of providing information on the incomplete observation of the Earth surface by the input sensor. This may be caused by a sensor failure or by persistent cloud coverage.
Number of patches: 
Number of contiguous groups of burned pixels. Contiguity is defined as any burned pixel that has contact with the side of another burned pixel during the month.

In [ ]:
df_burned = create_column(ds, df_burned, 3, 'standard_error')
df_burned = create_column(ds, df_burned, 3, 'fraction_of_burnable_area')
df_burned = create_column(ds, df_burned, 3, 'fraction_of_observed_area')
df_burned = create_column(ds, df_burned, 3, 'number_of_patches')
df_burned

,lon,lat,burned_area,no_data_burned_area,tree_cover_broadleaved_evergreen_burned_area,tree_cover_broadleaved_deciduous_burned_area,tree_cover_needleleaved_evergreen_burned_area,tree_cover_needleleaved_deciduous_burned_area,tree_cover_mixed_leave_burned_area,cropland_burned_area,mosaic_tree_grass_burned_area,other_vegetation_burned_area,flooded_area_burned_area,standard_error,fraction_of_burnable_area,fraction_of_observed_area,number_of_patches
0,-179.875,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-179.625,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-179.375,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-179.125,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-178.875,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1036795,178.875,-89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1036796,179.125,-89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1036797,179.375,-89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1036798,179.625,-89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Auswertung einzelner Data Frame

Muss bedacht werden: 
Fraction of burnable area: 
An example of this layer corresponding to August 2019 is shown in Figure 3.4. This layer does not change monthly, but only when a new land cover year is considered (see section 2.8)
Fraction of observed area:
Recommendation on product use: this is a very important attribute to consider, as it shows the proportion of each cell that was not observed in a particular month and therefore it identifies the regions where the product may miss burned pixels. All grid cells with fraction of observed area lower than 80% should be used with care.
Standard_error: unklar


In [25]:
df_burned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1036800 entries, 0 to 1036799
Data columns (total 17 columns):
 #   Column                                         Non-Null Count    Dtype  
---  ------                                         --------------    -----  
 0   lon                                            1036800 non-null  float32
 1   lat                                            1036800 non-null  float32
 2   burned_area                                    1036800 non-null  float32
 3   no_data_burned_area                            1036800 non-null  float32
 4   tree_cover_broadleaved_evergreen_burned_area   1036800 non-null  float32
 5   tree_cover_broadleaved_deciduous_burned_area   1036800 non-null  float32
 6   tree_cover_needleleaved_evergreen_burned_area  1036800 non-null  float32
 7   tree_cover_needleleaved_deciduous_burned_area  1036800 non-null  float32
 8   tree_cover_mixed_leave_burned_area             1036800 non-null  float32
 9   cropland_burned_area    

In [31]:
for column_name in df_burned.columns:
    column = df_burned[column_name]
    count = (column == 0).sum()
    print('Anzahl an 0 in Spalte', column_name, ' ist : ', count)

Anzahl an 0 in Spalte lon  ist :  0
Anzahl an 0 in Spalte lat  ist :  0
Anzahl an 0 in Spalte burned_area  ist :  1029171
Anzahl an 0 in Spalte no_data_burned_area  ist :  1033335
Anzahl an 0 in Spalte tree_cover_broadleaved_evergreen_burned_area  ist :  1035138
Anzahl an 0 in Spalte tree_cover_broadleaved_deciduous_burned_area  ist :  1032580
Anzahl an 0 in Spalte tree_cover_needleleaved_evergreen_burned_area  ist :  1036675
Anzahl an 0 in Spalte tree_cover_needleleaved_deciduous_burned_area  ist :  1036800
Anzahl an 0 in Spalte tree_cover_mixed_leave_burned_area  ist :  1036793
Anzahl an 0 in Spalte cropland_burned_area  ist :  1032481
Anzahl an 0 in Spalte mosaic_tree_grass_burned_area  ist :  1034957
Anzahl an 0 in Spalte other_vegetation_burned_area  ist :  1031717
Anzahl an 0 in Spalte flooded_area_burned_area  ist :  1035968
Anzahl an 0 in Spalte standard_error  ist :  1029171
Anzahl an 0 in Spalte fraction_of_burnable_area  ist :  798409
Anzahl an 0 in Spalte fraction_of_observ

### Erste Visualisierung 

In [23]:
#fig = go.Figure(data=go.Scattergeo(
#        lon = df_burned['lon'],
#        lat = df_burned['lat'],
#        marker = dict(
#            size = 8,
#            opacity = 0.8,
#            reversescale = True,
#            autocolorscale = False,
#            symbol = 'square',
#            line = dict(
#                width=1,
#                color='rgba(102, 102, 102)'
#            ),
#            colorscale = 'Blues',
#            cmin = df_burned['burned_area'].min(),
#            color = df_burned['burned_area'],
#            cmax = df_burned['burned_area'].max(),
#            colorbar_title="Burned Area Total"
#        )))

#fig.update_layout(
#        title = 'Burned Areas',
#    )
#fig.show()


Durch eine erste Visualisierung wird ein Überblick über die Daten erzielt und die Datenverarbeitung kann plausiblisiert werden. 

## Vorgehensweise

Folgende Schritte sollen innerhalb der Projektarbeit umgesetzt weren: 

1.	Dataframe (siehe [Erstellung Dataframe](#erstellung-dataframe)) um weitere Variablen aus dem ursprünglichen Daten erweitern, z.B. fraction_of_burnable_area
2.	Dataframe mit Attributen zu Kontinent und Land anreichern (z.B. mit Bibliothek ``shaply``)
3.	Überprüfung durchführen, ob weitere berechnete Attribute notwendig sind
4. (ggf.Attribute berechnen)
5.	Dataframe in SQL-Datenbank exportieren
6.	Zeitliche Dimension, durch die Durchführung der Schritt 1-6 für weitere Dateien, zur Datenbank ergänzen
7.	Erstellung verschiedener Abfragen auf die Datenbank
8.	Explorative Datenanalyse der Daten 


## Quellen

[1]: https://www.spiegel.de/politik/deutschland/sachsen-michael-kretschmer-bricht-urlaub-wegen-waldbraenden-ab-a-9cde7363-0b93-4a5a-a711-ff7c5391858e, Datum des Aufrufs: 07.08.2022

[2]: https://de.statista.com/infografik/18581/anzahl-und-flaeche-aller-waldbraende-in-deutschland/, Datum des Aufrufs: 07.08.2022

[3]: https://climate.esa.int/en/odp/#/dashboard, Datum des Aufrufs: 07.08.2022
